# 진수정 2017311974 / 최고 성능(accuracy): 

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# 한글 폰트 다운
!apt-get update -qq
!apt-get install fonts-nanum* -qq

Selecting previously unselected package fonts-nanum.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-6_all.deb ...
Unpacking fonts-nanum-eco (1.000-6) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20170925-1_all.deb ...
Unpacking fonts-nanum-extra (20170925-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-1_all.deb ...
Unpacking fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-extra (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Setting up fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-eco (1.000-6) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


> 데이터 불러오기

In [3]:
import numpy as np
import pandas as pd

# 첫번째 행이 비어있으며 두번째 행에 변수명이 적혀있으므로, header = 1로 설정
train = pd.read_csv("/content/gdrive/MyDrive/자연어처리/ttrain.csv",header = 1)
test = pd.read_csv("/content/gdrive/MyDrive/자연어처리/ttest.csv",header = 1)

In [4]:
train.head(5)

,title,label
0,유튜브 내달 2일까지 크리에이터 지원 공간 운영,IT과학
1,어버이날 맑다가 흐려져…남부지방 옅은 황사,생활문화
2,내년부터 국가RD 평가 때 논문건수는 반영 않는다,IT과학
3,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것,IT과학
4,회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간,생활문화


> 데이터 전처리 - 한글이 아닌 것 삭제

In [5]:
train['title'] = train['title'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ")
train['title'] = train['title'].str.replace(" +"," ")

train.dropna(inplace = True)
train.reset_index(drop = True,inplace = True)

test['title'] = test['title'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ")
test['title'] = test['title'].str.replace(" +"," ")

test.dropna(inplace = True)
test.reset_index(drop = True,inplace = True)

> 데이터 전처리 - 품사 태깅 후 명사, 동사, 형용사만 남기기

명사, 동사, 형용사로 뉴스의 카테고리를 유의미하게 판별할 수 있다고 판단하여, 해당 품사만 추출하였습니다.

In [6]:
import nltk
nltk.download('all',quiet = True)

True

In [7]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 56.0 MB/s 
     |████████████████████████████████| 86 kB 5.1 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [8]:
from konlpy.tag import Okt
okt = Okt()

In [9]:
target_pos = ['Noun','Verb','Adjective']

def filter_by_pos(text):
  text = okt.pos(text, stem = True, norm = True)
  cleaned = [w[0] for w in text if w[1] in target_pos]
  return " ".join(cleaned)

train['title'] = train['title'].map(filter_by_pos)
test['title'] = test['title'].map(filter_by_pos)

> 데이터 전처리 - 워드클라우드 생성 결과 이용하여 불용어 제거

워드클라우드를 생성하여 모든 카테고리에서 공통적으로 등장하는 단어를 확인한 후, 이를 불용어로 설정하여 제거하였습니다.

In [10]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [11]:
set(train['label'])

{'IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치'}

In [12]:
# 각 뉴스 카테고리에 해당하는 텍스트 저장
science = ''
economy = ''
society = ''
culture = ''
world = ''
sports = ''
politics = ''

for idx in range(len(train)):
  if train.loc[idx,'label'] == 'IT과학':
    science = ' '.join([science,train.loc[idx,'title']])
  elif train.loc[idx,'label'] == '경제':
    economy = ' '.join([economy,train.loc[idx,'title']])
  elif train.loc[idx,'label'] == '사회':
    society = ' '.join([society,train.loc[idx,'title']])
  elif train.loc[idx,'label'] == '생활문화':
    culture = ' '.join([culture,train.loc[idx,'title']])
  elif train.loc[idx,'label'] == '세계':
    world = ' '.join([world,train.loc[idx,'title']])
  elif train.loc[idx,'label'] == '스포츠':
    sports = ' '.join([sports,train.loc[idx,'title']])
  elif train.loc[idx,'label'] == '정치':
    politics = ' '.join([politics,train.loc[idx,'title']])

In [13]:
topics = [science, economy, society, culture, world, sports, politics]

plt.rcParams['figure.figsize'] = [60,100]

for idx in range(7):
  wordcloud = WordCloud(background_color='white',
                        width = 300,
                        height = 300,
                        font_path = '/usr/share/fonts/truetype/nanum/NanumSquareL.ttf').generate(topics[idx])
  plt.subplot(4,2,idx+1)
  plt.imshow(wordcloud,interpolation = 'bilinear')
  plt.axis('off')

plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [14]:
stop_words = ['종합','하다','이다','올해','내달','내년','내일','되다','없다','않다']

def remove_stopwords(text):
  text = okt.pos(text,stem = True,norm = True)
  cleaned = [w[0] for w in text if w[0] not in stop_words]
  return " ".join(cleaned)

train['title'] = train['title'].map(remove_stopwords)
test['title'] = test['title'].map(remove_stopwords)

> 클래스 불균형

사회 분야의 뉴스 데이터가 다른 분야보다 확연히 적은 것을 확인할 수 있었습니다.   
학습 데이터를 변형하는 방법이나 (SMOTE), 모델에 가중치를 적용하는 방법을 통해 클래스 불균형 문제를 해결하려고 합니다
.

In [15]:
print(train.label.value_counts() / len(train.label))

세계      0.175883
정치      0.174986
IT과학    0.171899
스포츠     0.168309
경제      0.131967
생활문화    0.131836
사회      0.045120
Name: label, dtype: float64


> 벡터화 : Tf-idf

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [17]:
X_train, y_train = train['title'], train['label']
X_test, y_test = test['title'], test['label']

In [18]:
tfidf = TfidfVectorizer(max_features = 16000,min_df = 2)
x_train = tfidf.fit_transform(X_train)
x_test = tfidf.transform(X_test)

> SMOTE + Naive Bayes

In [29]:
from imblearn.over_sampling import SMOTE
from sklearn.naive_bayes import MultinomialNB
from imblearn.pipeline import Pipeline, make_pipeline
from sklearn import metrics
from sklearn.metrics import classification_report
import itertools

In [20]:
smote = SMOTE(random_state = 123, k_neighbors = 3)
x_train_sm, y_train_sm = smote.fit_resample(x_train,y_train)

In [53]:
params = {'alpha':(0.0001,0.001,0.01,0.05,0.1,0.3,0.5,0.7,0.9,1)}
results = []

for alpha in itertools.product(params['alpha']):
  clf = MultinomialNB(alpha = alpha)
  clf.fit(x_train_sm, y_train_sm)
  pred = clf.predict(x_test)
  acc_val = metrics.accuracy_score(y_test, pred)
  results.append((alpha,acc_val))

results = pd.DataFrame(results)
results.columns = ['alpha','Accuracy']
results.sort_values(by = ['Accuracy'],ascending = False,axis = 0,inplace = True)
results

,alpha,Accuracy
9,"(1,)",0.726804
8,"(0.9,)",0.725486
7,"(0.7,)",0.724717
6,"(0.5,)",0.722851
5,"(0.3,)",0.715713
4,"(0.1,)",0.700340
3,"(0.05,)",0.689799
2,"(0.01,)",0.666959
1,"(0.001,)",0.643681
0,"(0.0001,)",0.630394


In [54]:
clf = MultinomialNB(alpha = 1)
clf.fit(x_train_sm, y_train_sm)
pred = clf.predict(x_test)
metrics.accuracy_score(y_test, pred)

0.726803557702866

> SMOTE + SVM

In [25]:
from sklearn.svm import SVC
import itertools

In [57]:
Cs = [0.1,1,10,100,1000]
gammas = [0.001,0.01,0.05,0.1,0.5,1]
kernels = ['rbf']
results = []

for C,gamma,kernel in itertools.product(Cs,gammas,kernels):
  clf = SVC(C = C,gamma = gamma,kernel = kernel)
  clf.fit(x_train_sm, y_train_sm)
  pred = clf.predict(x_test)
  acc_val = metrics.accuracy_score(y_test, pred)
  results.append((C, gamma, kernel, acc_val))

results = pd.DataFrame(results)
results.columns = ['C','gamma','kernel','Accuracy']
results.sort_values(by = ['Accuracy'],ascending = False,axis = 0,inplace = True)
results

,C,gamma,kernel,Accuracy
18,100.0,0.001,rbf,0.724168
13,10.0,0.010,rbf,0.716811
14,10.0,0.050,rbf,0.697705
24,1000.0,0.001,rbf,0.688152
19,100.0,0.010,rbf,0.686615
15,10.0,0.100,rbf,0.685956
8,1.0,0.050,rbf,0.680905
16,10.0,0.500,rbf,0.679807
22,100.0,0.500,rbf,0.677940
28,1000.0,0.500,rbf,0.677281


> Weighted SVM

In [22]:
print(len(train.label) / train.label.value_counts())

세계       5.685586
정치       5.714750
IT과학     5.817371
스포츠      5.941467
경제       7.577638
생활문화     7.585188
사회      22.163028
Name: label, dtype: float64


In [26]:
weight = {'세계':5.68, '정치':5.71, 'IT과학':5.81, '스포츠':5.94, '경제':7.57, '생활문화':7.58, '사회':22.16}

In [24]:
Cs = [0.1,1,10,100,1000]
gammas = [0.001,0.01,0.05,0.1,0.5,1]
kernels = ['rbf']
results = []

for C,gamma,kernel in itertools.product(Cs,gammas,kernels):
  clf = SVC(C = C,gamma = gamma,kernel = kernel,class_weight = weight)
  clf.fit(x_train, y_train)
  pred = clf.predict(x_test)
  acc_val = metrics.accuracy_score(y_test, pred)
  results.append((C, gamma, kernel, acc_val))

results = pd.DataFrame(results)
results.columns = ['C','gamma','kernel','Accuracy']
results.sort_values(by = ['Accuracy'],ascending = False,axis = 0,inplace = True)
results

,C,gamma,kernel,Accuracy
12,10.0,0.001,rbf,0.744153
7,1.0,0.010,rbf,0.743823
3,0.1,0.100,rbf,0.739541
2,0.1,0.050,rbf,0.730098
8,1.0,0.050,rbf,0.730098
4,0.1,0.500,rbf,0.722302
9,1.0,0.100,rbf,0.708356
13,10.0,0.010,rbf,0.708246
18,100.0,0.001,rbf,0.707917
10,1.0,0.500,rbf,0.680905


In [27]:
clf = SVC(C = 10,gamma = 0.001,kernel = 'rbf',class_weight = weight)
clf.fit(x_train, y_train)
pred = clf.predict(x_test)
metrics.accuracy_score(y_test, pred)

0.7441528494564621

In [35]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

        IT과학       0.51      0.78      0.62       738
          경제       0.84      0.73      0.78      1701
          사회       0.86      0.66      0.75      3356
        생활문화       0.71      0.86      0.78      1367
          세계       0.66      0.73      0.69       739
         스포츠       0.90      0.86      0.88       565
          정치       0.58      0.83      0.68       641

    accuracy                           0.74      9107
   macro avg       0.72      0.78      0.74      9107
weighted avg       0.77      0.74      0.75      9107



> SMOTE + Random Forest

In [22]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold,StratifiedKFold

In [24]:
clf = RFC(random_state = 123)
clf.fit(x_train_sm,y_train_sm)
pred = clf.predict(x_test)
metrics.accuracy_score(y_test, pred)

0.5977819259909959

In [25]:
param_grid = {
    'n_estimators':[200,400,600,800,1000],
    'max_depth':[2,4,6],
    'min_samples_split':[4,8,12,16,20],
    'min_samples_leaf':[2,6,10]
}

In [31]:
rf_grid = GridSearchCV(estimator = RFC(random_state = 123), param_grid = param_grid,
                       cv = KFold(n_splits = 5, random_state = 42,shuffle = True), 
                       n_jobs = -1, verbose = 2)
rf_grid.fit(x_train_sm, y_train_sm)
rf_grid.best_params_

Fitting 5 folds for each of 225 candidates, totalling 1125 fits


{'max_depth': 6,
 'min_samples_leaf': 2,
 'min_samples_split': 8,
 'n_estimators': 1000}

In [ ]:
rf_grid.best_score_

In [23]:
best_rf = RFC(random_state = 123,
              max_depth = 6,
              min_samples_leaf = 2,
              min_samples_split = 8,
              n_estimators = 1000)
best_rf.fit(x_train_sm,y_train_sm)
pred = best_rf.predict(x_test)
metrics.accuracy_score(y_test, pred)

0.5023608213462172

> LSTM

In [24]:
import random
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding,Conv1D, MaxPooling1D,Flatten
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec

In [125]:
random.seed(42)
np.random.seed(42)
os.environ["PYTHONHASHSEED"] = str(42)

In [166]:
X_train, y_train = train['title'], train['label']
X_test, y_test = test['title'], test['label']

In [167]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [168]:
label, cnt = np.unique(y_train,return_counts = True)
dict(zip(label,cnt))

{0: 7852, 1: 6028, 2: 2061, 3: 6022, 4: 8034, 5: 7688, 6: 7993}

In [169]:
weight2 = dict(zip(label,len(train.label) / cnt))
weight2

{0: 5.817371370351503,
 1: 7.577637690776377,
 2: 22.16302765647744,
 3: 7.585187645300564,
 4: 5.685586258401792,
 5: 5.941467221644121,
 6: 5.71475040660578}

In [170]:
sample_weight = np.ones(shape = (len(y_train),))
sample_weight[y_train == 2] = 4.5

In [171]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [172]:
tokenizer = Tokenizer(num_words = 10000)
tokenizer.fit_on_texts(X_train)
x_train = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen = 100)
x_test = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen = 100)

In [173]:
model = Sequential()
model.add(Embedding(10000, 100))
model.add(LSTM(128))
model.add(Dense(7, activation='softmax'))
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 2)
mc = ModelCheckpoint('best_model.h5', monitor = 'val_acc', mode = 'max', verbose = 2, save_best_only = True)
model.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['acc'])
history = model.fit(x_train, y_train, 
                    batch_size = 40, epochs = 30, 
                    callbacks = [es, mc],validation_split = 0.2,
                    sample_weight = sample_weight)

Epoch 1/30
914/914 [==============================] - ETA: 0s - loss: 1.1636 - acc: 0.6845
Epoch 00001: val_acc improved from -inf to 0.89602, saving model to best_model.h5
914/914 [==============================] - 136s 146ms/step - loss: 1.1636 - acc: 0.6845 - val_loss: 0.4752 - val_acc: 0.8960
Epoch 2/30
914/914 [==============================] - ETA: 0s - loss: 0.5389 - acc: 0.8630
Epoch 00002: val_acc improved from 0.89602 to 0.90915, saving model to best_model.h5
914/914 [==============================] - 132s 144ms/step - loss: 0.5389 - acc: 0.8630 - val_loss: 0.4020 - val_acc: 0.9092
Epoch 3/30
914/914 [==============================] - ETA: 0s - loss: 0.4163 - acc: 0.8951
Epoch 00003: val_acc improved from 0.90915 to 0.91780, saving model to best_model.h5
914/914 [==============================] - 132s 145ms/step - loss: 0.4163 - acc: 0.8951 - val_loss: 0.3754 - val_acc: 0.9178
Epoch 4/30
914/914 [==============================] - ETA: 0s - loss: 0.3612 - acc: 0.9083
Epoch 000

In [174]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(x_test, y_test)[1]))

285/285 [==============================] - 9s 29ms/step - loss: 0.9557 - acc: 0.6979

 테스트 정확도: 0.6979


> Word Embedding + LSTM

In [ ]:
random.seed(42)
np.random.seed(42)
os.environ["PYTHONHASHSEED"] = str(42)

In [ ]:
X_train, y_train = train['title'], train['label']
X_test, y_test = test['title'], test['label']

In [ ]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [ ]:
label, cnt = np.unique(y_train,return_counts = True)
dict(zip(label,cnt))

In [ ]:
weight2 = dict(zip(label,len(train.label) / cnt))
weight2

In [ ]:
sample_weight = np.ones(shape = (len(y_train),))
sample_weight[y_train == 2] = 4.5

In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
vocab_size = 10000
tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(X_train)
x_train = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen = 100)
x_test = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen = 100)

In [ ]:
sg_model = Word2Vec(size = 100,window = 5,min_count = 10,workers = 8,sg = 1,iter = 12)

In [ ]:
sg_model.build_vocab(X_train)

In [ ]:
sg_model.train(X_train,total_examples = 8000,epochs = 128)

In [ ]:
embed_mat = np.zeros((vocab_size, 100))

for word, idx in tokenizer.word_index.items():
  if word in sg_model.wv:
    embed_mat[idx] = sg_model.wv[word]

print(embed_mat.shape)

In [ ]:
embed_layer = Embedding(vocab_size, 100, weights=[embed_mat], input_length = 100, trainable = False)